In [22]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model


In [23]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [24]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.keras')

In [25]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [26]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [27]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [28]:
def get_response(intents_list, intents__json):
    tag = intents_list[0]['intent']
    list_of_intents = intents__json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [29]:
while True:
    message = input('You: ')
    if message == 'exit':
        break
    ints = predict_class(message)
    res = get_response(ints, intents)   
    print('Bot:', res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Bot: Hi there, how can I help?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Bot: Hi there, how can I help?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Bot: Hi there, how can I help?
